#Продвинутый Python, ДЗ-2

Правила игры:

В домашке 8 задач, разбаловка указана в задании. Суммарно за дз можно получить 100 баллов, что равняется 10 баллам

В каждой задаче необходимо реализовать функцию, которая после будет проверяться через github classroom на тестах. Сами тесты лежит в гитхабе, можете локально проверить работу функций перед сдачей

Дедлайн - 7 дней после выдачи дз. Необходимо залить решеннный ноутбук в github и прислать ссылку в Anytask (без выполнения любого из пунктов работа проверяться не будет)

В данной домашке нужно использовать pandas

In [43]:
# https://stackoverflow.com/a/60658965/7286121

from IPython.core.magic import register_cell_magic

@register_cell_magic
def write_and_run(line, cell):
    argz = line.split()
    file = argz[-1]
    mode = 'w'
    if len(argz) == 2 and argz[0] == '-a':
        mode = 'a'
    with open(file, mode) as f:
        f.write(cell)
    get_ipython().run_cell(cell)

## Данные

В этом домашнем задании вам придется оказаться на месте аналитика в бразильском маркетплейсе [Olist](https://olist.com/pt-br/). Вам необходимо исследовать данные и на их основании сделать выводы, которые помогут бизнесу расцветать!

Данные находятся тут в файле archive.zip

In [38]:
!pip install wget
#перед сдачей это закомментить

In [3]:
# Код, который будет в каждом тесте, названия не менять
import pandas as pd
import numpy as np
import zipfile
import wget

url = 'https://github.com/Palladain/Deep_Python/raw/main/Homeworks/Homework_1/archive.zip'
filename = wget.download(url)

with zipfile.ZipFile(filename, 'r') as zip_ref:
    zip_ref.extractall('./')

customers = pd.read_csv('olist_customers_dataset.csv')
location = pd.read_csv('olist_geolocation_dataset.csv')
items = pd.read_csv('olist_order_items_dataset.csv')
payments = pd.read_csv('olist_order_payments_dataset.csv')
reviews = pd.read_csv('olist_order_reviews_dataset.csv')
orders = pd.read_csv('olist_orders_dataset.csv')
products = pd.read_csv('olist_products_dataset.csv')
translation = pd.read_csv('product_category_name_translation.csv')
sellers = pd.read_csv('olist_sellers_dataset.csv')

Ван дано 9 датасетов, которые содержат в себе все данные по 100 000 заказам со всей Бразилии. Чтобы облегчить вам жизнь, вот связи по этим датасетам (файл product_category_name_translation является переводом названий категорий с португальского на английский)

![](https://i.imgur.com/HRhd2Y0.png)

Ну что же, начнем!

## Задание 1 (10 баллов)

Определите:

* Число товаров

* Среднюю стоимость товара (стоимость товара = среднее от цен в датасете items)

в разрезе категорий (все категории должны быть на английском языке)

Табличка, которая у вас должна получиться:

```
category | products | price

value    | value      | value
```

**Обратите внимание:**

Для категории portateis_cozinha_e_preparadores_de_alimentos перевод portable kitchen and food preparers

Для категории pc_gamer перевод PC Gamer

Для них нужно отдельно добавить перевод

In [109]:
#from itertools import product
%%write_and_run task_1.py

# важно! все зависимости, которые используете (если добавляее новые) в этом классе надо явно продублировать в эту ячейку
import numpy as np
import pandas as pd

def task_1(translation, items, products):
    products = products.copy(deep = True)

    translated = translation.set_index("product_category_name")["product_category_name_english"].to_dict()
    translated["portateis_cozinha_e_preparadores_de_alimentos"] = "portable kitchen and food preparers"
    translated["pc_gamer"] = "PC Gamer"
    products.dropna(axis = 0, subset = ["product_category_name"], inplace = True)
    products["translated_category"] = products["product_category_name"].apply(lambda x: translated[x])

    table = items.groupby("product_id")
    table1 = table.agg({"price": "mean"}).reset_index()
    table2 = pd.merge(products, table1, on = "product_id")
    result = table2.groupby("translated_category").agg({"product_id": "count", "price": "mean"})
    result.reset_index(inplace = True)
    result.columns = ["category", "products", "price"]
    return result
    pass

In [110]:
#Проверки

res = task_1(translation, items, products)

assert res[res.category == 'portable kitchen and food preparers'].price.values[0] == 186.996
assert len(res) == 73
assert len(res.drop_duplicates()) == 73
assert res[res.category == 'drinks'].products.values[0] == 81
assert res.products.sum() == 32341
assert res.price.sum() == 12459.751444351941
assert res[res.category == 'home_confort'].price.values[0] == 185.56926417326417

## Задание 2 (10 баллов)

Определите для каждого продавца основную категорию их продаж (категории должны быть на английском языке)

Табличка, которая у вас должна получиться:

```
seller_id | category

value    | value
```




In [111]:
%%write_and_run task_2.py

# важно! все зависимости, которые используете (если добавляее новые) в этом классе надо явно продублировать в эту ячейку
import numpy as np
import pandas as pd

def task_2(translation, products, items):
    added_translations = pd.DataFrame([["pc_gamer", "PC Gamer"],
                                      ["portateis_cozinha_e_preparadores_de_alimentos", "portable kitchen and food preparers"]], columns=translation.columns)
    translation = pd.concat([translation, added_translations])
    merged = items.join(products.set_index("product_id"), on="product_id")
    merged = merged.join(translation.set_index("product_category_name"), on="product_category_name")
    seller_main_category = merged.groupby(["seller_id", "product_category_name_english"]).agg({"order_id": "count"}).reset_index()
    seller_main_category = seller_main_category.sort_values("order_id", ascending=False, axis = 0).drop_duplicates(["seller_id"])
    seller_main_category = seller_main_category.drop(columns=["order_id"], axis=1)
    seller_main_category = seller_main_category.rename(columns={"product_category_name_english": "category"})

    return seller_main_category



In [112]:
# Проверки

res = task_2(translation, products, items)
assert res[res.seller_id == 'e3e15e2c0b9700561efac21c6be48066'].category.values[0] == 'housewares'
assert res[res.seller_id == '2f73e04d12cdf0c945ded66bb3fcf6c7'].category.values[0] == 'garden_tools'
assert len(res) == len(res.drop_duplicates())
assert len(res) == 3035
assert len(res[res.category == 'telephony']) == 66
assert list(np.sort(res.groupby("category").agg({"seller_id": "nunique"}).seller_id.values)) == [  1,   1,   1,   1,   1,   2,   2,   2,   2,   2,   3,   3,   4,
         4,   4,   4,   5,   5,   5,   5,   5,   5,   6,   6,   6,   7,
         8,  10,  12,  13,  13,  13,  14,  14,  14,  15,  16,  17,  17,
        17,  19,  20,  20,  20,  21,  22,  26,  37,  37,  43,  46,  51,
        54,  59,  66,  66,  78,  87,  87,  99, 101, 116, 125, 156, 216,
       224, 256, 288, 310]

## Задание 3 (10 баллов)

Выведите долю потраченных денег по каждому штату (потраченные деньги - сумма денег по доставленным заказам, сумма по price и freight_value)

*Примечание:* разбивка по штатам - по штату покупателя, процент - число от 0 до 1


Табличка, которая у вас должна получиться:

```
state | perc

value | value
```

In [113]:
%%write_and_run task_3.py

# важно! все зависимости, которые используете (если добавляее новые) в этом классе надо явно продублировать в эту ячейку
import numpy as np
import pandas as pd

def task_3(orders, customers, items):
    orders_items = pd.merge(orders, items, on="order_id", how ="left")
    merged_data = pd.merge(orders_items, customers, on="customer_id", how="left")
    delivered_orders = merged_data[merged_data['order_status'] == 'delivered']
    delivered_orders['total_spent'] = delivered_orders['price'] + delivered_orders['freight_value']
    spent_in_state = delivered_orders.groupby('customer_state')['total_spent'].sum().reset_index()
    total_spent = spent_in_state['total_spent'].sum()
    spent_in_state['perc'] = spent_in_state['total_spent'] / total_spent
    spent_in_state.rename(columns={'customer_state': 'state'}, inplace=True)
    return (spent_in_state[['state', 'perc']])
    pass

In [114]:
# Проверки

res = task_3(orders, customers, items)
assert res.perc.sum() == 1
assert res[res.state == "RS"].perc.values[0] == 0.055868056429816286
assert res.sort_values("perc", ascending=True).iloc[0, 1] == 0.0005862290943146945
assert res.sort_values("perc", ascending=False).iloc[0, 1] == 0.3741756035817322
assert len(res) == 27

<ipython-input-113-32837119e675>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  delivered_orders['total_spent'] = delivered_orders['price'] + delivered_orders['freight_value']


## Задание 4 (10 баллов)

Определите средний чек покупки (добавьте разбивку на стоимость самого заказ и стоимость доставки) и среднее число товаров в заказе

А также определите среднее число покупок на пользователя (обратите внимание на идентификаторы)

In [115]:
%%write_and_run task_4.py

# важно! все зависимости, которые используете (если добавляее новые) в этом классе надо явно продублировать в эту ячейку
import numpy as np
import pandas as pd

def task_4(items, orders, customers):
    items_agg = items.groupby("order_id").agg({
        "price": "sum",
        "freight_value": "sum",
        "order_item_id": "max"
    })

    # Расчет среднего чека, средней стоимости доставки, среднего количества товаров в заказе
    avg_cheque = items_agg["price"].mean()
    avg_cheque_delivery = items_agg["freight_value"].mean()
    avg_num_of_goods = items_agg["order_item_id"].mean()

    # среднее количество покупок на пользователя
    avg_num_of_purchase = customers.groupby("customer_unique_id").agg({
        "customer_id": "count"
    })["customer_id"].mean()

    return avg_cheque, avg_cheque_delivery, avg_num_of_goods, avg_num_of_purchase



In [116]:
# Проверка

res = task_4(items, orders, customers)
assert res == (137.7540763788945, 22.823561713254815, 1.1417306873695092, 1.0348089410589412)

## Задание 5 (10 баллов)

Посчитайте CSAT (customer satisfaction - средняя оценка ревью) и отобразите средний CSAT по дням в период с апреля 2017 по апрель 2018 года

Все манипуляции с датой необходимо сделать с помощью datetime и dateutil


Табличка, которая у вас должна получиться:

```
date | csat

"YYYY-MM-DD" | value
```

In [117]:
%%write_and_run task_5.py

# важно! все зависимости, которые используете (если добавляее новые) в этом классе надо явно продублировать в эту ячейку
import numpy as np
import pandas as pd
from dateutil import parser
from dateutil.parser import parse
import datetime

def task_5(reviews):

    reviews["review_creation_date"] = pd.to_datetime(reviews["review_creation_date"])
    reviews["date"] = reviews["review_creation_date"].dt.date #создала столбец без времени, только дата
    begin = pd.to_datetime('2017-04-01').date()
    end = pd.to_datetime("2018-04-30").date()
    reviews_sorted = reviews[(reviews["date"] >= begin) & (reviews["date"] <= end)] #данные за нужный период
    csat = reviews_sorted.groupby("date")["review_score"].mean().reset_index()
    csat.columns = ["date", "csat"]
    csat["date"] = csat["date"].apply(lambda x: x.strftime("%Y-%m-%d"))
    return csat
    pass

In [118]:
# Проверки
res = task_5(reviews)
assert res.date.min() == '2017-04-01'
assert res.date.max() == '2018-04-30'
assert res.csat.sum() == 1551.8881071384853
assert res[res.date == '2017-07-11'].csat.values[0] == 4.291390728476821
assert res[res.date == '2018-02-09'].csat.values[0] == 3.992156862745098
assert res[res.csat == 3.6814814814814816].date.values[0] == '2018-02-25'

## Задание 6 (10 баллов)

Посмотрите, как быстро отвечают пользователи (сделайте аггреграцию по числу дней ответа) и какая средняя оценка

Все манипуляции со временем нужно делать через datetime и dateutil

Табличка, которая у вас должна получиться:

```
days | csat | orders

value | value | value
```

Результаты должны быть отсориртированы по дня по возрастанию

In [119]:
%%write_and_run task_6.py

# важно! все зависимости, которые используете (если добавляее новые) в этом классе надо явно продублировать в эту ячейку
import numpy as np
import pandas as pd
from dateutil.parser import parse
import datetime

def task_6(reviews):
    reviews["review_creation_date"] = pd.to_datetime(reviews["review_creation_date"])
    reviews["review_answer_timestamp"] = pd.to_datetime(reviews["review_answer_timestamp"])
    reviews["answer_time"] = (reviews["review_answer_timestamp"] - reviews["review_creation_date"]).dt.days
    result = reviews.groupby("answer_time").agg({"review_score": "mean", "order_id": "count"}).reset_index()
    result.columns = ["days", "csat", "orders"]
    return result
    # YOUR CODE
    pass

In [120]:
# Проверки
res = task_6(reviews)
assert res.orders.sum() == 99224
assert np.all(res.days.values == np.sort(res.days.values))
assert len(res) == 214
assert res.days.min() == 0
assert res.days.max() == 518
assert res[res.days == 233].csat.values[0] == 3.0
assert res[res.days == 87].orders.values[0] == 4

## Задание 7 (10 баллов)

Выделите все заказы, где не проставлено поле order_delivered_customer_date. Замените его на дату '2999-12-31'

In [121]:
%%write_and_run task_7.py

# важно! все зависимости, которые используете (если добавляее новые) в этом классе надо явно продублировать в эту ячейку
import numpy as np
import pandas as pd

def task_7(orders):
    orders["order_delivered_customer_date"].fillna('2999-12-31', inplace = True)
    return orders
    pass

In [122]:
#Проверки

res = task_7(orders)
assert len(res[res.order_delivered_customer_date.isna()]) == 0
assert len(res) == 99441
assert len(res[res.order_delivered_customer_date == '2999-12-31']) == 2965

## Задание 8 (30 баллов)

Определите топ-10 продавцов, у которых больше 100 заказов, отсылающие чаще всего свою посылку в другие регионы (считаются только доставленные заказы)

Чаще всего отсылают = самый большой процент отправленных заказов в другой штат

Табличка, которая у вас должна получиться:

```
seller_id | share

value | value
```

In [123]:
%%write_and_run task_8.py

# важно! все зависимости, которые используете (если добавляее новые) в этом классе надо явно продублировать в эту ячейку
import numpy as np
import pandas as pd

def task_8(orders, items, sellers, customers):
    merged_data = items.merge(orders, on='order_id').merge(sellers, on='seller_id').merge(customers, on='customer_id')
    merged_data.drop_duplicates

    delivered_orders = merged_data[merged_data['order_status'] == 'delivered'] # Фильтруем только доставленные заказы
    delivered_orders['is_to_other_region'] = delivered_orders['seller_state'] != delivered_orders['customer_state']   #новый столбец, true если не равны, false если один штат
    grouped_data = delivered_orders.groupby('seller_id') #группируем по продавцам
    total_delivered_orders = grouped_data.size()
    total_other_region_orders = grouped_data['is_to_other_region'].sum()
    share = (total_other_region_orders / total_delivered_orders).fillna(0)
    top_sellers = share[total_delivered_orders >= 100].nlargest(10)
    return top_sellers.reset_index().rename(columns={'seller_id': 'seller', 0: 'share'})

    pass

In [125]:
# Проверки

res = task_8(orders, items, sellers, customers)
assert np.all(res.share.values == np.sort(res.share.values)[::-1])
assert res.share.values[0] == 0.9743589743589743
assert res.share.values[-1] == 0.9356435643564357
assert res.seller_id.values[5] == '1b4c3a6f53068f0b6944d2d005c9fc89'
assert res.seller_id.values[2] == '06a2c3af7b3aee5d69171b0e14f0ee87'
assert res.share.sum() == 9.519118744616716

<ipython-input-123-0e10f64d5929>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  delivered_orders['is_to_other_region'] = delivered_orders['seller_state'] != delivered_orders['customer_state']   #новый столбец, true если не равны, false если один штат


AssertionError: ignored